<a href="https://colab.research.google.com/github/drscook/day1_materials/blob/main/day1_materials_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# run once to update packages
import google
! pip install -U ipython-autotime numpy pandas matplotlib plotly_express
google.colab.output.clear() # clear messy output from update process
get_ipython().kernel.do_shutdown(True) # restart runtime so updates take effect - ignore "session crashed" pop-up

{'status': 'ok', 'restart': True}

In [2]:
import warnings, numpy as np, pandas as pd
from IPython.core.display import HTML

### helper functions ###
def to_numeric(df):
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=(FutureWarning,UserWarning))
        def func(ser):
            dt = str(ser.dtype).lower()
            if 'geometry' not in dt and 'bool' not in dt and 'category' not in dt:
                ser = pd.to_numeric(ser.astype('string').str.lower().str.strip().replace('',pd.NA), downcast='integer', errors='ignore')
                if pd.api.types.is_string_dtype(ser):
                    ser = pd.to_datetime(ser, errors='ignore')
                elif pd.api.types.is_integer_dtype(ser):
                    ser = ser.astype('Int64', errors='ignore')
            return ser
        return pd.DataFrame(df).apply(func).convert_dtypes()

def prep(df):
    rename_column = lambda x: x.lower().replace(' ','_').replace('-','_') if isinstance(x, str) else x
    df = pd.DataFrame(df).rename(columns=rename_column).rename_axis(index=rename_column)
    idx = pd.MultiIndex.from_frame(df[[]].reset_index().to_numeric())
    return df.to_numeric().set_index(idx).rename_axis(df.index.names)

def disp(df, max_rows=4, max_cols=200, **kwargs):
    display(HTML(pd.DataFrame(df).to_html(max_rows=max_rows, max_cols=max_cols, **kwargs)))

for func in [to_numeric, prep, disp]:
    for cls in [pd.DataFrame, pd.Series]:
        setattr(cls, func.__name__, func)

### start ###
raw = pd.read_csv('https://raw.githubusercontent.com/drscook/day1_materials/main/data/day1_materials_raw.csv')
raw = raw.rename(columns=raw.iloc[0]).iloc[2:]
col_rename = {
    'response_id': 'id',
    'recorded_date': 'date',
    'duration_(in_seconds)': 'duration',
    'progress': 'progress',
    'your_classification': 'class',
    'primary_location': 'campus',
    'do_you_have_internet_access_at_home_that_is_stable_and_fast_enough_to_effectively_use_online_course_materials?': 'internet',
    'on_the_first_class_day,_did_you_have_all_required_materials_for_all_your_courses?': 'first',
    'today,_do_you_have_all_required_materials_for_all_your_courses?': 'today',
    'if_not,_why?_(check_all_that_apply_or_skip_if_you_answered_yes_above)___selected_choice': 'why',
    'which_of_these_do_you_own_and_use_for_classes_(check_all_that_apply)?': 'electronics',
    'where_did_you_get_your_course_materials_(check_all_that_apply)?___selected_choice': 'sources',
    'how_much_did_you_spend_on_required_materials?___$': 'spend',
    'how_many_of_your_courses_require_materials_that_you_pay_for_(textbooks,_equipment,_online_services,_software,_etc)?': 'num_crse_reported_pay',
    'how_many_courses_are_you_taking?': 'num_crse_reported',
    }
R = raw.prep().rename(columns=col_rename).set_index('id').sort_index()
A = R.filter(col_rename.values())

C = R.filter(regex='[0-9]__')
crse_cols = ['subj', 'numb', 'require', 'first', 'today', 'use']
L = len(crse_cols)
K = len(C.columns) // L
C.columns = crse_cols * K

F = pd.read_csv('https://raw.githubusercontent.com/drscook/day1_materials/main/data/course_fix.csv')
C = (
    pd.concat([C.iloc[:,i*L:(i+1)*L] for i in range(K)])
    .dropna(how='all')
    .reset_index()
    .merge(F[['subj', 'numb', 'numb new']], how='left')
    .assign(numb = lambda x: x['numb new'])
    .drop(columns='numb new')
    .prep()
    .set_index('id')
    .sort_index()
)

repl = {
    'require': {
        'no required materials': '0_no_required',
        'required materials are all free': '1_free_required',
        'yes': '2_yes_required'
        },
    'first': {
        'none': '0_none_first',
        'some but not all': '1_some_first',
        'most but not all': '2_most_first',
        'yes': '3_all_first',
        'i had no materials': '0_none_first',
        'i had some but not all materials': '1_some_first',
        'i had most but not all materials': '2_most_first',
        'i had all material': '3_all_first'},
    'today': {
        'none': '0_none_today',
        'some but not all': '1_some_today',
        'most but not all': '2_most_today',
        'yes': '3_all_today',
        'i had no materials': '0_none_today',
        'i had some but not all materials': '1_some_today',
        'i had most but not all materials': '2_most_today',
        'i had all material': '3_all_today'},
    'campus':{
        'stephenville':'steph',
        'fort worth':'ftw',
        'online/distance learner':'dist',
    },
    'class': {
        'high school':'0_hs',
        'other': '0_other',
        'freshman': '1_fr',
        'sophomore': '2_so',
        'junior': '3_jr',
        'senior': '4_sr',
        'graduate': '5_gr',
    },
    'internet': {
        'no': '0_no_internet',
        'yes': '1_yes_internet',
    },
    'use': {
        'never': '0_never_use',
        'occasionally': '1_occasionally_use',
        'often': '2_often_use',
    },
}


for k, v in repl.items():
    if k in A:
        A[k] = A[k].replace(v)
    if k in C:
        C[k] = C[k].replace(v)

A['num_crse_details_pay'] = C.query('require==100').groupby('id').size()
A['num_crse_details'] = C.groupby('id').size()
A['num_crse_pay'] = A[['num_crse_reported_pay', 'num_crse_details_pay']].max(axis=1)
A['num_crse'] = A[['num_crse_reported', 'num_crse_details', 'num_crse_pay']].max(axis=1)
A['spend_per_crse_pay'] = (A['spend'] / A['num_crse_pay']).clip(0, 500)
A['spend_per_crse'] = (A['spend'] / A['num_crse']).clip(0, 500)
A['pct_crse_pay'] = A['num_crse_pay'] / A['num_crse'] * 100
labeler = lambda s: pd.get_dummies(s.str.split(',').explode(), prefix=s.name).groupby('id').sum().astype('boolean')
cs = ['campus','electronics','sources','why']
B = pd.concat([labeler(A[k]) for k in cs], axis=1)
cs.remove('campus')
A = A.join(B, how='outer').drop(columns=cs).round().prep().sort_index()
C = C.join(A, lsuffix='_crse').prep().sort_index()

A.to_csv('students.csv')
C.to_csv('courses.csv')
with pd.ExcelWriter('processed.xlsx') as writer:
    raw.to_excel(writer, sheet_name='raw')
    A.to_excel(writer, sheet_name='students')
    C.to_excel(writer, sheet_name='courses')
    F.to_excel(writer, sheet_name='courses_fix')

<ipython-input-2-6586ead6601c>:130: RuntimeWarning: Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.
  A['num_crse_details_pay'] = C.query('require==100').groupby('id').size()
